In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import cv2
import datetime
import dateutil.relativedelta
import math

In [8]:
ts = datetime.datetime.fromtimestamp(1619195309.272862)
ts2 = datetime.datetime.fromtimestamp(1619195677.208)
diff = dateutil.relativedelta.relativedelta(ts2, ts)
print(diff)

relativedelta(minutes=+6, seconds=+7, microseconds=+935138)


In [28]:
def draw_xywha(im, num, x, y, w, h, angle, color=(0,255,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    font = cv2.FONT_HERSHEY_SIMPLEX
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w, -h], [w, -h], [w, h], [-w, h]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([2*x, 2*y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    cv2.putText(im,str(num),(int(2 *x)-15, int(2 *y)), font, 2, (0, 255, 0), 3, cv2.LINE_AA)

def drawing_bb(folder_path, video_name, RP1, RP2):

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v') ,5, (1024, 1024))
    allFiles = os.listdir(folder_path)
    jsonList = []
    files = []
    RP = RP1 + RP2
    RP = sorted(RP)
    RP_index = 0
    RP = [f for f in RP if '._' not in f]
    print(RP)
    for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
        if '.json' in f:
            if '._' not in f:
                jsonList.append(f)
#     print(files)
    jsonList = sorted(jsonList)
    files = sorted(files)
#     print(len(jsonList), len(files))
    RP1count = 0
    RP2count = 0
    for i in range(len(files)):
        loc =  folder_path + files[i]
        jloc = folder_path + jsonList[i]
        with open(jloc) as f:
            cam1 = json.load(f)
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.imread(loc)
        boxes = list(cam1["Bounding Box"].keys())
        count = 0
        for b in boxes:
            count+=1
            draw_xywha(image, count, cam1["Bounding Box"][b][0], cam1["Bounding Box"][b][1], cam1["Bounding Box"][b][2], cam1["Bounding Box"][b][3], cam1["Bounding Box"][b][4], color=(0,255,0), linewidth=5)
#             cv2.putText(image,str(count),(int(cam1["Bounding Box"][b][0])-15, int(cam1["Bounding Box"][b][1])), font, 2, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image,"People#: " + str(cam1["Num People"]),(1600, 50), font, 2, (0, 0, 255), 3, cv2.LINE_AA)
        ts = datetime.datetime.fromtimestamp(1619195309.272862)
        ts2 = datetime.datetime.fromtimestamp(float(cam1['Timestamp']))
        diff = dateutil.relativedelta.relativedelta(ts2, ts)
        print(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60), (diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60)    
        cv2.putText(image,"since: " + str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60))+"min "+str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60))+"sec",(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        
        RP_relative = float(RP[RP_index].replace('.json', ''))
        if float(cam1['Timestamp']) > RP_relative:
            if RP[RP_index] in RP1:
                with open('/home/team19/Fusion/RP1/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP1count = int(RPinfo['count'])
            elif RP[RP_index] in RP2:
                with open('/home/team19/Fusion/RP2/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP2count = int(RPinfo['count'])
            if RP_index < len(RP)-1:
                RP_index += 1
        print("TDS1: " + str(RP1count), "TDS2: " + str(RP2count))
        cv2.putText(image, "TDS1: " + str(RP2count),(1600, 1900), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.putText(image, "TDS2: " + str(RP1count),(1600, 1950), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        resized = cv2.resize(image, (1024,1024))
        out.write(resized)
        
#     for data_file in sorted(files):
#         loc = folder_path+data_file
#         print(loc)
#         image = cv2.imread(loc)
        
#         resized = cv2.resize(image, (1024,1024))

#         out.write(resized)

        
    out.release()

In [29]:
folder_path = '/home/team19/Desktop/Axis_DL/Detection/YOLO/04-23-2021-12:28:27/Camera 1/'
video_name = 'trial.mp4'
RP1 = os.listdir('/home/team19/Fusion/RP1/')
RP2 = os.listdir('/home/team19/Fusion/RP2/')
drawing_bb(folder_path, video_name, RP1, RP2)

['1619195677.208.json', '1619195792.834.json', '1619195867.016.json', '1619195924.93.json', '1619196871.52.json', '1619196957.225.json', '1619197017.935.json', '1619197100.482.json']
0 0
TDS1: 0 TDS2: 0
0 1
TDS1: 0 TDS2: 0
0 3
TDS1: 0 TDS2: 0
0 5
TDS1: 0 TDS2: 0
0 7
TDS1: 0 TDS2: 0
0 9
TDS1: 0 TDS2: 0
0 11
TDS1: 0 TDS2: 0
0 13
TDS1: 0 TDS2: 0
0 15
TDS1: 0 TDS2: 0
0 16
TDS1: 0 TDS2: 0
0 18
TDS1: 0 TDS2: 0
0 20
TDS1: 0 TDS2: 0
0 22
TDS1: 0 TDS2: 0
0 24
TDS1: 0 TDS2: 0
0 26
TDS1: 0 TDS2: 0
0 28
TDS1: 0 TDS2: 0
0 30
TDS1: 0 TDS2: 0
0 32
TDS1: 0 TDS2: 0
0 34
TDS1: 0 TDS2: 0
0 35
TDS1: 0 TDS2: 0
0 37
TDS1: 0 TDS2: 0
0 39
TDS1: 0 TDS2: 0
0 41
TDS1: 0 TDS2: 0
0 43
TDS1: 0 TDS2: 0
0 45
TDS1: 0 TDS2: 0
0 47
TDS1: 0 TDS2: 0
0 49
TDS1: 0 TDS2: 0
0 51
TDS1: 0 TDS2: 0
0 52
TDS1: 0 TDS2: 0
0 54
TDS1: 0 TDS2: 0
0 56
TDS1: 0 TDS2: 0
0 58
TDS1: 0 TDS2: 0
1 0
TDS1: 0 TDS2: 0
1 2
TDS1: 0 TDS2: 0
1 4
TDS1: 0 TDS2: 0
1 6
TDS1: 0 TDS2: 0
1 8
TDS1: 0 TDS2: 0
1 10
TDS1: 0 TDS2: 0
1 12
TDS1: 0 TDS2: 0
1 13
TDS1

12 15
TDS1: 0 TDS2: 4
12 17
TDS1: 0 TDS2: 4
12 19
TDS1: 0 TDS2: 4
12 21
TDS1: 0 TDS2: 4
12 23
TDS1: 0 TDS2: 4
12 25
TDS1: 0 TDS2: 4
12 27
TDS1: 0 TDS2: 4
12 28
TDS1: 0 TDS2: 4
12 30
TDS1: 0 TDS2: 4
12 32
TDS1: 0 TDS2: 4
12 34
TDS1: 0 TDS2: 4
12 36
TDS1: 0 TDS2: 4
12 38
TDS1: 0 TDS2: 4
12 40
TDS1: 0 TDS2: 4
12 42
TDS1: 0 TDS2: 4
12 44
TDS1: 0 TDS2: 4
12 46
TDS1: 0 TDS2: 4
12 48
TDS1: 0 TDS2: 4
12 50
TDS1: 0 TDS2: 4
12 52
TDS1: 0 TDS2: 4
12 54
TDS1: 0 TDS2: 4
12 55
TDS1: 0 TDS2: 4
12 57
TDS1: 0 TDS2: 4
12 59
TDS1: 0 TDS2: 4
13 1
TDS1: 0 TDS2: 4
13 3
TDS1: 0 TDS2: 4
13 5
TDS1: 0 TDS2: 4
13 7
TDS1: 0 TDS2: 4
13 9
TDS1: 0 TDS2: 4
13 11
TDS1: 0 TDS2: 4
13 13
TDS1: 0 TDS2: 4
13 15
TDS1: 0 TDS2: 4
13 17
TDS1: 0 TDS2: 4
13 19
TDS1: 0 TDS2: 4
13 21
TDS1: 0 TDS2: 4
13 23
TDS1: 0 TDS2: 4
13 25
TDS1: 0 TDS2: 4
13 26
TDS1: 0 TDS2: 4
13 28
TDS1: 0 TDS2: 4
13 30
TDS1: 0 TDS2: 4
13 32
TDS1: 0 TDS2: 4
13 34
TDS1: 0 TDS2: 4
13 36
TDS1: 0 TDS2: 4
13 38
TDS1: 0 TDS2: 4
13 40
TDS1: 0 TDS2: 4
13 42
TDS1: 0 T

24 29
TDS1: 0 TDS2: 4
24 31
TDS1: 0 TDS2: 4
24 33
TDS1: 0 TDS2: 4
24 35
TDS1: 0 TDS2: 4
24 37
TDS1: 0 TDS2: 4
24 39
TDS1: 0 TDS2: 4
24 41
TDS1: 0 TDS2: 4
24 42
TDS1: 0 TDS2: 4
24 44
TDS1: 0 TDS2: 4
24 46
TDS1: 0 TDS2: 4
24 48
TDS1: 0 TDS2: 4
24 50
TDS1: 0 TDS2: 4
24 52
TDS1: 0 TDS2: 4
24 54
TDS1: 0 TDS2: 4
24 56
TDS1: 0 TDS2: 4
24 58
TDS1: 0 TDS2: 4
25 0
TDS1: 0 TDS2: 4
25 2
TDS1: 0 TDS2: 4
25 4
TDS1: 0 TDS2: 4
25 6
TDS1: 0 TDS2: 4
25 8
TDS1: 0 TDS2: 4
25 10
TDS1: 0 TDS2: 4
25 12
TDS1: 0 TDS2: 4
25 13
TDS1: 0 TDS2: 4
25 15
TDS1: 0 TDS2: 4
25 17
TDS1: 0 TDS2: 4
25 19
TDS1: 0 TDS2: 4
25 21
TDS1: 0 TDS2: 4
25 23
TDS1: 0 TDS2: 4
25 25
TDS1: 0 TDS2: 4
25 27
TDS1: 0 TDS2: 4
25 29
TDS1: 0 TDS2: 4
25 31
TDS1: 0 TDS2: 4
25 33
TDS1: 0 TDS2: 4
25 35
TDS1: 0 TDS2: 4
25 37
TDS1: 0 TDS2: 4
25 39
TDS1: 0 TDS2: 4
25 41
TDS1: 0 TDS2: 4
25 43
TDS1: 0 TDS2: 4
25 45
TDS1: 0 TDS2: 4
25 46
TDS1: 0 TDS2: 4
25 48
TDS1: 0 TDS2: 4
25 50
TDS1: 0 TDS2: 4
25 52
TDS1: 0 TDS2: 4
25 54
TDS1: 0 TDS2: 4
25 56
TDS1: 0 T